<div style="font-size: 14pt;">Prof. Krzysztof Rybinski</div><br/><br/>
<div style="font-size: 22pt;"><b>Artificial Intelligence course</b></div><br/><br/>
<div style="font-size: 18pt;">LAB 10</div><br/>
<div style="font-size: 18pt;">- RMP dataset</div><br/>
<div style="font-size: 18pt;">- emotion classification</div><br/>
<div style="font-size: 18pt;">- zero-shot classification</div><br/>

In [1]:
import torch

In [2]:
#exetute this cell only in Google Colab
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [3]:
from transformers import pipeline
import os
import torch
import numpy as np
import pandas as pd
import pickle

In [ ]:
#exetute this cell only in Google Colab
from google.colab import files
import io

In [88]:
#get index for the maximum score from prediction in model distilbert-base-uncased-emotion
############################################################
def get_max_index(ld):
    lv = ld['scores']
    lvind = lv.index(max(lv))
    return lvind
############################################################

In [5]:
#get label with the maximum score for model distilbert-base-uncased-mnli
############################################################
def get_max_label(ld):
    val = ld['scores']
    val_ind = lv.index(max(lv))
    label = ld['labels'][val_ind]
    return label
############################################################

In [6]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
rmp = pd.read_csv("https://raw.githubusercontent.com/nindaba/semister-5-ai/main/lab-9/rmp_small.csv") #for local drive

In [ ]:
rmp

,review,score
0,"he can be a mean man sometimes, but just get o...",2
1,"He's fun, but he speeds through lectures and l...",2
2,Dr. Foster is an amazing teacher. One of my fa...,4
3,She is one of the greatest teacher I have ever...,4
4,Professor Bird was very easy and relaxed. If y...,3
...,...,...
97540,Charismatic and enthusiastic but that's it . H...,0
97541,"Great books - Dickens, Eliot, etc. A little d...",3
97542,Awesome professor. Doesnt stop explaining unti...,4
97543,"He has a really high standard, but grades fair...",4


In [8]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [9]:
#for the final project skip this line
sentences = sentences[0:5000]
labels = labels[0:5000]

In [10]:
#check documentation https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True, device=0)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [11]:
#check emotion prdictions
prediction = classifier(sentences[0:10])

In [12]:
prediction[1], sentences[1]

([{'label': 'sadness', 'score': 0.017645377665758133},
  {'label': 'joy', 'score': 0.2766183614730835},
  {'label': 'love', 'score': 0.0019553741440176964},
  {'label': 'anger', 'score': 0.635624349117279},
  {'label': 'fear', 'score': 0.06618060171604156},
  {'label': 'surprise', 'score': 0.001975938444957137}],
 "He's fun, but he speeds through lectures and lessons like a madman. If you have not taken Spanish before, you definitely will have to study.")

In [13]:
prediction[6], sentences[6]

([{'label': 'sadness', 'score': 0.001150374417193234},
  {'label': 'joy', 'score': 0.4458451569080353},
  {'label': 'love', 'score': 0.5510021448135376},
  {'label': 'anger', 'score': 0.0007522721425630152},
  {'label': 'fear', 'score': 0.0004594257625285536},
  {'label': 'surprise', 'score': 0.0007906428654678166}],
 "A very easy grader. Best advice would be to go to her after class and speak with her to get to know her on a personal level...that'll help you more in the class. She's a sweet professor with rich worldly experiences that often relate to class materials.")

In [15]:
prediction = classifier(sentences)

In [16]:
len(prediction)

5000

In [81]:
 l_max_scores = [get_max_index(val) for val in prediction]

TypeError: ignored

In [79]:
l_max_scores[0:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
prediction[1]

[{'label': 'sadness', 'score': 0.017645377665758133},
 {'label': 'joy', 'score': 0.2766183614730835},
 {'label': 'love', 'score': 0.0019553741440176964},
 {'label': 'anger', 'score': 0.635624349117279},
 {'label': 'fear', 'score': 0.06618060171604156},
 {'label': 'surprise', 'score': 0.001975938444957137}]

In [21]:
#convert scores to labels
aaa = l_max_scores
aaa = ["sadness" if x == 0 else x for x in aaa]
aaa = ["joy" if x == 1 else x for x in aaa]
aaa = ["love" if x == 2 else x for x in aaa]
aaa = ["anger" if x == 3 else x for x in aaa]
aaa = ["fear" if x == 4 else x for x in aaa]
aaa = ["surprise" if x == 5 else x for x in aaa]
aaa[0:10]


['joy', 'anger', 'joy', 'joy', 'joy', 'joy', 'love', 'anger', 'sadness', 'joy']

In [23]:
df = pd.DataFrame(list(zip(aaa, labels)), columns =['emotion', 'score'])
df

,emotion,score
0,joy,2
1,anger,2
2,joy,4
3,joy,4
4,joy,3
...,...,...
4995,joy,3
4996,joy,4
4997,anger,1
4998,joy,0


In [24]:
#calculate crosstabulations
result = pd.crosstab(df["emotion"], df["score"])
result

score,0,1,2,3,4
emotion,,,,,
anger,356,153,133,151,134
fear,33,24,24,23,18
joy,201,207,381,924,1444
love,5,6,19,57,93
sadness,211,103,84,55,56
surprise,6,9,16,28,46


In [25]:
#do zero-shot classification
#check documentation https://huggingface.co/typeform/distilbert-base-uncased-mnli
classifier = pipeline("zero-shot-classification", 
                      model = "typeform/distilbert-base-uncased-mnli",
                      device = 0 
)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [26]:
l1 = "poor or bad professor teaching skills"
l2 = "unfair or improper grading"
l3 = "poor or bad professor interpersonal skills or qualities"
l4 = "extremely or very difficult tests or exams"
l5 = "harassment or abuse or improper behavior"
l6 = "course or professor review or opinion"
candidate_labels=[l1, l2, l3, l4, l5, l6]

In [27]:
len(sentences)

5000

In [28]:
prediction = classifier(sentences, candidate_labels=candidate_labels)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7facb6ca5790> was reported to be 5000 (when accessing len(dataloader)), but 27501 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7facb6ca5790> was reported to be 5000 (when accessing len(dataloader)), but 27502 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7facb6ca5790> was reported to be 5000 (when accessing len(dataloader)), but 27503 samples have been fetched. 
  warnings.warn(warn_msg

In [29]:
prediction[90], prediction[101]

({'sequence': 'Frustrating. Most of the work was filler, and there wasn\'t much substance to the class. Some people call it "weeding-out" uninterested folk, I call it lack of organization.',
  'labels': ['poor or bad professor interpersonal skills or qualities',
   'unfair or improper grading',
   'harassment or abuse or improper behavior',
   'poor or bad professor teaching skills',
   'course or professor review or opinion',
   'extremely or very difficult tests or exams'],
  'scores': [0.4011043608188629,
   0.1886785477399826,
   0.1557510495185852,
   0.10986804962158203,
   0.09110179543495178,
   0.05349617078900337]},
 {'sequence': "Such a boring class, incredibly easy. If you're at all interested in economics don't take, very disappointed. Ended up with an A, yet didn't learn anything. The entire course is geared to spewing conservative trickle down economics facts. Someone could essentially miss every class and if you went into the exams with a republican agenda, easy A.",
  

In [30]:
lv = prediction[90]['scores']
lv

[0.4011043608188629,
 0.1886785477399826,
 0.1557510495185852,
 0.10986804962158203,
 0.09110179543495178,
 0.05349617078900337]

In [32]:
lvind = lv.index(max(lv))
lvind

0

In [40]:
l_max_labels = [get_max_label(val) for val in prediction]

In [34]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['prof_opinion', 'score'])
df

,prof_opinion,score
0,course or professor review or opinion,2
1,course or professor review or opinion,2
2,course or professor review or opinion,4
3,course or professor review or opinion,4
4,course or professor review or opinion,3
...,...,...
4995,extremely or very difficult tests or exams,3
4996,course or professor review or opinion,4
4997,course or professor review or opinion,1
4998,poor or bad professor teaching skills,0


In [35]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

score,0,1,2,3,4
prof_opinion,,,,,
course or professor review or opinion,151,120,237,603,1125
extremely or very difficult tests or exams,113,131,152,218,154
harassment or abuse or improper behavior,41,16,12,15,13
poor or bad professor interpersonal skills or qualities,241,126,169,306,397
poor or bad professor teaching skills,195,49,40,28,36
unfair or improper grading,71,60,47,68,66


# Final project

1. Propose new classification, each group proposes its own </br>
2. Choose two zero shot models from https://huggingface.co/models?pipeline_tag=zero-shot-classification&language=en&sort=downloads </br>
3. Calculate predictions for each model on Google Colab</br>
4. Calculate cross-tabulation for each prediction </br>
5. Compare results, are there any differences </br>
6. Write conclusions about difference, speed of computation etc.</br>
7. Write what the results tell us about the student opinions regarding courses and professors </br>
8. Are there any recommendations for universities' management </br>
9. Make a presentation and send together with Jupyter Notbook to rybinski@rybinski.eu by Wednesday, 12 April 2023, 23:59
<br/><br/>
<div style="font-size: 16pt;">Grading rules</div><br/>

1. Project done in student teams <br/>
2. You can get max 4 points <br/>
3. 1 point for the classification and its justification <br/>
4. 1 point for the correct code for each model
5. 1 point for the summary, lessons learned and presentation clarity <br/>


In [36]:
classifier = pipeline("zero-shot-classification", 
                      model = "cross-encoder/nli-deberta-v3-large",
                      device = 0
)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [37]:
predition = classifier(sentences, candidate_labels=candidate_labels)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7facb6cfd160> was reported to be 5000 (when accessing len(dataloader)), but 27501 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7facb6cfd160> was reported to be 5000 (when accessing len(dataloader)), but 27502 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7facb6cfd160> was reported to be 5000 (when accessing len(dataloader)), but 27503 samples have been fetched. 
  warnings.warn(warn_msg

In [39]:
prediction[0]

{'sequence': 'he can be a mean man sometimes, but just get on his good side. overall hes an easy teacher to take. midterm/final is running a mile, but if u have health probs he will let u off easy. u work out on ur own.',
 'labels': ['course or professor review or opinion',
  'unfair or improper grading',
  'harassment or abuse or improper behavior',
  'poor or bad professor interpersonal skills or qualities',
  'extremely or very difficult tests or exams',
  'poor or bad professor teaching skills'],
 'scores': [0.22883568704128265,
  0.22512812912464142,
  0.17601191997528076,
  0.14763721823692322,
  0.1265450417995453,
  0.09584203362464905]}

In [89]:
#modify if necessary
# l_max_scores = [get_max_index(val) for val in prediction]
label = [get_max_label(item) for item in prediction]

In [90]:
#modify if necessary
#convert scores to labels
# aaa = l_max_scores
# aaa = ["poor or bad professor teaching skills" if x == 0 else x for x in aaa]
# aaa = ["unfair or improper grading" if x == 1 else x for x in aaa]
# aaa = ["poor or bad professor interpersonal skills or qualities" if x == 2 else x for x in aaa]
# aaa = ["extremely or very difficult tests or exams" if x == 3 else x for x in aaa]
# aaa = ["harassment or abuse or improper behavior" if x == 4 else x for x in aaa]
# aaa = ["course or professor review or opinion" if x == 5 else x for x in aaa]
# aaa[0:10]

['poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills',
 'poor or bad professor teaching skills']

In [93]:
df = pd.DataFrame(list(zip(label, labels)), columns =['prof_opinion', 'score'])
df

,prof_opinion,score
0,course or professor review or opinion,2
1,course or professor review or opinion,2
2,course or professor review or opinion,4
3,course or professor review or opinion,4
4,course or professor review or opinion,3
...,...,...
4995,extremely or very difficult tests or exams,3
4996,course or professor review or opinion,4
4997,course or professor review or opinion,1
4998,poor or bad professor teaching skills,0


In [94]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

score,0,1,2,3,4
prof_opinion,,,,,
course or professor review or opinion,151,120,237,603,1125
extremely or very difficult tests or exams,113,131,152,218,154
harassment or abuse or improper behavior,41,16,12,15,13
poor or bad professor interpersonal skills or qualities,241,126,169,306,397
poor or bad professor teaching skills,195,49,40,28,36
unfair or improper grading,71,60,47,68,66
